In [ ]:
'''
'''

In [190]:
import socket
import copy
import time
import math
import crcmod #change for own implementation

#  Functions

In [191]:
def get_CRC(mess):
    #ver la tabla del modulo crcmod para ver porque es 0x11021 y demas
    #g(x)=x^16+x^12+x^5+1 -- > 0x11021--> 10001000000100001b
    crc16_func = crcmod.mkCrcFun(0x11021, initCrc=0xFFFF, xorOut=0x0000,rev=False)
    crc16val = crc16_func(mess)#valor de crc16
    crcbyte1 = (crc16val & 65280) >>8
    crcbyte2 = crc16val & 255
    return crcbyte1,crcbyte2,crc16val


# Read data

## PMU parameters

In [192]:
#idcode de la PMU
idcode=1
tcpip= "10.10.200.22"
tcpport = 4712
buffersize= 2048


## Ask for configuration frame

In [193]:
#if you use wireshark, you need to ask first for the cfg2 frame
#and mantain the socket connection open. wireshark uses this information
# to decode the dataframes
#send get cfg2 command
# 1 field SYNC 
field1_1 = 170#AA.
field1_2 = 65 #0-reservado100-data frame comando0001-2005
#2 FRAMESIZE
field2_1 =0  #el tamaño siempre es 18 incluyendo los dos bits del CHK
field2_2=18
# 3 field IDCODE
#dividir en dos bytes y hacer un bitshift 8 lugares a la derecha
field3_1 = idcode >> 8 
field3_2 = idcode & 255 #quedaria 00000001 para un idcode=1
#4 SOC
#send unix time
PCtime = time.time()
t =int(PCtime).to_bytes(4,'big')
field4_1 =t[0]
field4_2 =t[1]
field4_3 =t[2]
field4_4 =t[3]
#5 FRACSEC
#fmod:get fractional part,
#round: to convert fractional part to 3 bytes (precision is lost)
#str:to do slicing and remove "0." part
#int to_bytes: list of 3 bytes of the fractional part
fsec = int(str(round(math.fmod(PCtime,1),7))[2:]).to_bytes(3,'big')
field5_1 =0 #time quality flags, ignore leap seconds
field5_2 =fsec[0]
field5_3 =fsec[1]
field5_4 =fsec[2]
#6 CMD
field6_1 = 0 #primer byte es cero
field6_2= 5 #00000101- comandoget CFG2
#7 EXTFRAME no se usa
#8 CHK
#mensajae sin CHK
messnoCHK = [field1_1 , field1_2,
    field2_1,field2_2,
    field3_1, field3_2,
    field4_1, field4_2, field4_3, field4_4,
    field5_1, field5_2, field5_3, field5_4,
    field6_1, field6_2]

[crcbyte1,crcbyte2,crc_16] = get_CRC(bytearray(messnoCHK));
#mensaje con los datos del CFG2 que seran enviados
message = messnoCHK.copy()
message.append(crcbyte1)
message.append(crcbyte2)
message_bytearr = bytearray(message)
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
s.connect((tcpip, tcpport))
s.send(message_bytearr)
data = s.recv(buffersize)

print('Received raw configuration frame:')
print (data)
#print every byte received with the byte number and binary format
#print('Datos en forma de lista:')
#for i,d in enumerate(list(data)):
#    print('byte {} - {} - {} - {}'.format(i,d, bin(d),chr(data[i])))

Received raw configuration frame:
b'\xaa1\x02z\x00\x01\x00\x00\\\xa6\x0f\x0fB@\x00\x98\x96\x80\x00\x01NI STATION      \x00\x01\x00\x0f\x00\x08\x00\x08\x00\x01V2+             VA2             IA2             VB2             IB2             VC2             IC2             IN2             P1              S1              Q1              PF1             P2              S2              Q2              PF2             DI0             DI1             DI2             DI3             DI4             DI5             DI6             DI7                                                                                                                                             \x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\n\x07\xdf'


## Send read data command

In [194]:

#Send turn on transmission of data frames and read 10 dataframes
# 1 field SYNC 
field1_1 = 170#AA.
field1_2 = 65 #0-reservado100-data frame comando0001-2005
#2 FRAMESIZE
#it's always 18 because im not using the EXTFRAME data (table 12-command frame configuration)
field2_1 =0  
field2_2=18
# 3 field IDCODE
#dividir en dos bytes y hacer un bitshift 8 lugares a la derecha
field3_1 = idcode >> 8 
field3_2 = idcode & 255 #quedaria 00000001 para un idcode=1
#4 SOC
#send unix time
PCtime = time.time()
t =int(PCtime).to_bytes(4,'big')
field4_1 =t[0]
field4_2 =t[1]
field4_3 =t[2]
field4_4 =t[3]
#5 FRACSEC
#fmod:get fractional part,
#round: to convert fractional part to 3 bytes (precision is lost)
#str:to do slicing and remove "0." part
#int to_bytes: list of 3 bytes of the fractional part
fsec = int(str(round(math.fmod(PCtime,1),7))[2:]).to_bytes(3,'big')
field5_1 =0 #time quality flags, ignore leap seconds
field5_2 =fsec[0]
field5_3 =fsec[1]
field5_4 =fsec[2]
#6 CMD
#0010 -turn on transmission of data frames.
field6_1 = 0 #first byte is zero
field6_2= 2
#7 EXTFRAME no se usa
#8 CHK
#mensajae sin CHK
messnoCHK = [field1_1 , field1_2,
    field2_1,field2_2,
    field3_1, field3_2,
    field4_1, field4_2, field4_3, field4_4,
    field5_1, field5_2, field5_3, field5_4,
    field6_1, field6_2]

[crcbyte1,crcbyte2,crc_16] = get_CRC(bytearray(messnoCHK));
#mensaje con los datos del CFG2 que seran enviados
message = messnoCHK.copy()
message.append(crcbyte1)
message.append(crcbyte2)
message_bytearr = bytearray(message)
s.send(message_bytearr)


Framesize: 124, length of dataframe: 124
b'\xaa\x01\x00|\x00\x01\x00\x00\\\xa6\x0f-\xc6\xc0!\xb0=TH\xa4?\x08\xbd\x85>\x1ed`?\x0bU\x8c;\x94\xb8\x00\xc0%\xcez:\xdf\xc0\x00\xc0%\x14\xc1;\xaf\xa0\x00\xc0%\x80\xc29\x14\xc0\x00>\xd3q\x0f;7\x00\x00\xc0%0\xc2:\xb5\x10\x00?\x02e\xb5BrU-@\xa9.\xe02\x98\xf4x4k5\xc2\xb4jn\x7f=\xa6yz\xba5\xbb\x07:5\xbc\x1e6\x9f=x\xbf\x7f\xfew\x00\xffw\x16'
Dataframe number: 1
------------------------------------------------------------
Framesize: 124, length of dataframe: 124
b'\xaa\x01\x00|\x00\x01\x00\x00\\\xa6\x0f=\t\x00!\xb0=T3E?\x0e\x1b=>\x1eT\xa0?\x10\xb3p;\x94\x8c\x00\xc0$uI:\xdf\xc8\x00\xc0#\xc1I;\xaf\xd0\x00\xc0$-I9\x14\xc0\x00>\xdal\xd5;6\xf8\x00\xc0#\xd1H:\xb3\xf0\x01?\x08\x03\x9bBm\x08\x07\xc0\xb9\x00j4\xe5\xdd\xaf5\x02FE\xb4uU\xfa?a\xdaI\xba5q\x7f:5r\x9f6\xa1\x92\x82\xbf\x7f\xfej\x00\xffB\xa0'
Dataframe number: 2
------------------------------------------------------------
Framesize: 124, length of dataframe: 124
b'\xaa\x01\x00|\x00\x01\x00\x00\\\xa6\x

In [ ]:
#read  measurements 10 times
#this code is not optimal, n frames per second are expected
#is only for testing purposes, data could be lost or superimposed
n= 0
dataframenumber = 0 #counter for dataframes
while n  < 10:
    #can only recieve 4096 bytes
    #maybe i should use a function that can get me 
    #the numbers of bytes before using the recv function
    data = s.recv(4096)
    #check if first byte is AA (begining of the dataframe)
    if data[:1].hex() =='aa':
        #get the framesize
        framesizebytes  = data[2:4]
        framesize = int.from_bytes(framesizebytes,'big')
        #the dataframes could be superimposed, so i get the framesize and
        #divide the data into n framesize strings
        #split data in chunks of framsize
        if len(data) > framesize:
            datachunks = [data[i:i+n] for i in range(0, len(data), framesize)]
            for d in datachunks:
                print('Framesize: {}, length of dataframe: {}'.format(framesize,len(d)))
                print(d)
                dataframenumber +=1
        else: #no superimposed dataframes
            print('Framesize: {}, length of dataframe: {}'.format(framesize,len(data)))
            print(data)
            dataframenumber +=1
        print('Dataframe number: {}'.format(dataframenumber))
        print('------------------------------------------------------------')

    
#     print(len(data))
#     print('-----------------------------------------------------------------')
#     print(data.hex().upper())
    

    n=n+1
    
#have to execute next cell to close socket

##  send command to turn off transmission (have to execute to close connection)


In [195]:

    

#Send turn on transmission of data frames and read 10 dataframes
# 1 field SYNC 
field1_1 = 170#AA.
field1_2 = 65 #0-reservado100-data frame comando0001-2005
#2 FRAMESIZE
#it's always 18 because im not using the EXTFRAME data (table 12-command frame configuration)
field2_1 =0  
field2_2=18
# 3 field IDCODE
#dividir en dos bytes y hacer un bitshift 8 lugares a la derecha
field3_1 = idcode >> 8 
field3_2 = idcode & 255 #quedaria 00000001 para un idcode=1
#4 SOC
#send unix time
PCtime = time.time()
t =int(PCtime).to_bytes(4,'big')
field4_1 =t[0]
field4_2 =t[1]
field4_3 =t[2]
field4_4 =t[3]
#5 FRACSEC
#fmod:get fractional part,
#round: to convert fractional part to 3 bytes (precision is lost)
#str:to do slicing and remove "0." part
#int to_bytes: list of 3 bytes of the fractional part
fsec = int(str(round(math.fmod(PCtime,1),7))[2:]).to_bytes(3,'big')
field5_1 =0 #time quality flags, ignore leap seconds
field5_2 =fsec[0]
field5_3 =fsec[1]
field5_4 =fsec[2]
#6 CMD
#0010 -turn off transmission of data frames.
field6_1 = 0 #first byte is zero
field6_2= 1
#7 EXTFRAME no se usa
#8 CHK
#mensajae sin CHK
messnoCHK = [field1_1 , field1_2,
    field2_1,field2_2,
    field3_1, field3_2,
    field4_1, field4_2, field4_3, field4_4,
    field5_1, field5_2, field5_3, field5_4,
    field6_1, field6_2]

[crcbyte1,crcbyte2,crc_16] = get_CRC(bytearray(messnoCHK));
#mensaje con los datos del CFG2 que seran enviados
message = messnoCHK.copy()
message.append(crcbyte1)
message.append(crcbyte2)
message_bytearr = bytearray(message)

s.close()